# TCSS503 - Week 7 Dynamic Programming

In this interactive tutorial we will review how dynamic programming can be used to solve computationally complex problems by building up solutions to smaller problems and using those calculations to solve harder problems.  We will use three different examples of increasing complexity to help with understanding.

## Fibonacci Sequence
  
The Fibonacci Sequence is a series of numbers where each number in the series is the sum of the previous two numbers.  The first and second numbers in the series are 0 and 1 respectively.

$fib(0) = 0$ \
$fib(1) = 1$ \
$fib(n) = fib(n-2) + fib(n-1)$

### Recursive Implementation
Instincts may be to implement this as a recursive function as the definition is itself recursive. Let's see what that code would look like.

In [1]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)
    
import time
for i in range(10):
    t_start = time.perf_counter()
    print(f"fib({i}) Results:{fib(i)}, Duration {time.perf_counter() - t_start:.2f} seconds")

fib(0) Results:0, Duration 0.00 seconds
fib(1) Results:1, Duration 0.00 seconds
fib(2) Results:1, Duration 0.00 seconds
fib(3) Results:2, Duration 0.00 seconds
fib(4) Results:3, Duration 0.00 seconds
fib(5) Results:5, Duration 0.00 seconds
fib(6) Results:8, Duration 0.00 seconds
fib(7) Results:13, Duration 0.00 seconds
fib(8) Results:21, Duration 0.00 seconds
fib(9) Results:34, Duration 0.00 seconds


## Results

We can see that this calcualtes the numbers correctly, and relatively quickly.  But the function starts to slow down once we reach numbers **less than 50!!!**

In [2]:

for i in range(40):
    t_start = time.perf_counter()
    print(f"fib({i}) Results:{fib(i)}, Duration {time.perf_counter() - t_start:.2f} seconds")

fib(0) Results:0, Duration 0.00 seconds
fib(1) Results:1, Duration 0.00 seconds
fib(2) Results:1, Duration 0.00 seconds
fib(3) Results:2, Duration 0.00 seconds
fib(4) Results:3, Duration 0.00 seconds
fib(5) Results:5, Duration 0.00 seconds
fib(6) Results:8, Duration 0.00 seconds
fib(7) Results:13, Duration 0.00 seconds
fib(8) Results:21, Duration 0.00 seconds
fib(9) Results:34, Duration 0.00 seconds
fib(10) Results:55, Duration 0.00 seconds
fib(11) Results:89, Duration 0.00 seconds
fib(12) Results:144, Duration 0.00 seconds
fib(13) Results:233, Duration 0.00 seconds
fib(14) Results:377, Duration 0.00 seconds
fib(15) Results:610, Duration 0.00 seconds
fib(16) Results:987, Duration 0.00 seconds
fib(17) Results:1597, Duration 0.00 seconds
fib(18) Results:2584, Duration 0.00 seconds
fib(19) Results:4181, Duration 0.00 seconds
fib(20) Results:6765, Duration 0.00 seconds
fib(21) Results:10946, Duration 0.01 seconds
fib(22) Results:17711, Duration 0.01 seconds
fib(23) Results:28657, Duration 

KeyboardInterrupt: 

## (Basic) Dynamic Programming to the rescue

This slow down is because the number of recursive calls to solve this problem is exponential:  ~$O(2^n)$  

This is due to the fact that each resursive call is having to solve the same problem over and over again.   We can do better by simply storing the values in an array of the previously calculated results.

In [ ]:
def fib_dp(n):
    if n < 2:
        return n
    solution = [0] * (n+1)
    solution[0] = 0
    solution[1] = 1
    for i in range(2,n+1):
        solution[i] = solution[i-1] + solution[i-2]

    return solution[-1]

import time
for i in range(500):
    t_start = time.perf_counter()
    print(f"fib_dp({i}) Results:{fib_dp(i)}, Duration {time.perf_counter() - t_start:.2f} seconds")

## Dynamic Programming Implementation Results
You can see we just calculated 500 values in a fraction of the time it took to calculate even one tenth of that using resursion.   This isn't to say that all recursion is bad, just when you have to solve the same problem over and over and over again.

## Faster Implementation?
Yes, there is a closed form of Fib that can be sovled in constant time.  It uses irrational numbers which can cause headaches with our finite computer processors (can't wait for infinite memory!) so for some values the rounding can cause issues.  There are solutions to that, but we won't going to cover those in this notebook.

## A look into what is happening...
It should be pretty clear with this toy example but to make things clear, we will take a look at the "solution" variable at each step and show what it returns at the end.  

In [ ]:
def fib_dp(n):
    if n < 2:
        return n
    solution = [0] * (n+1)
    solution[0] = 0
    solution[1] = 1
    for i in range(2,n+1):
        solution[i] = solution[i-1] + solution[i-2]
        print(f"i={i}:  {solution}")

    return solution[-1]


fib_dp(15)

## What we are seeing above...
Is the subsequent addition of a new value to the solution.  It is initialized with 0's (this is arbitrary, it could be any value).  And we see that at each step we are adding a new value that is the sum of the previous two values.

## Wasted Memory?
Yes, imagine if we were solving fib(100000).  We'd have a large array that we don't really need. This is simply an example of how dynamic programming works on a simple and easy to understand example.  In reality we needn't store a full array, but simply need to store the previous two values and shift them left as we iterate through, but that wouldn't be any fun in a dynamic programming notebook would it.  Plus most of the time when people are wanting the Fib series, they want all numbers 0...n not just the n'th element. :D

-----

# Job Selection

Given a list of consecutive values, find the highest sum of values such that no two selected numbers are contiguous (next to each other) in the array.

**Description of problem from the slides:**
- You are a wealthy socialite who charges for public appearances.  This month/year/life you have offers for appearances every day with different payouts.
- The problem is they are all in different cities in North America so if you work one job you cannot work jobs in the day before or after due to travel.
- You want to select the jobs to work to make the most money this month/year/life.

## Algorithm Concept
We want to build the solution from left to right.  We ask ourselves, on day n, what is the max $ I can earn.

- On Day 1 it is easy, take the job.
- On Day 2 it is also easy, take the job that is most valuable.
- On Day $n>2$ we have to say... do we take the job or not.
  - If we take the job, we can't have taken the day before's job.
  - If we don't take the job, we can.
  
So we say, we want to take the max of the n-2 day plus taking the job on day n... or the value of n-1.

In [ ]:
def job_selection(calendar):
    size = len(calendar)
    j = [0] * size
    s = [False] * size
    j[0] = calendar[0]
    s[0] = True
    j[1] = max(calendar[0], calendar[1])
    s[1] = calendar[1] > calendar[0]

    for i in range(2, size):
        with_last = j[i-2] + calendar[i]
        with_out_last = j[i-1]
        j[i] = max(with_last, with_out_last)
        s[i] = with_last > with_out_last
        
    return j


p = [15, 46, 43, 51, 92, 72, 61, 41, 39, 40, 82, 79, 42, 51]
job_selection(p)
    

## What does this show us?

Reading through this array, we can see that on day 1 the best is to take the first job.
Day 2 it is best to give up the job on day one and take the job on day 2.
On day 3... it looks like we take the job on day 1 and 3... but how do we know?  Human intuition.

We also see that on the end of the calendar we have solved our problem, and the maximum is $383 dollars.

Great, but what are the actual jobs from the array P that we selected????

Look at the values 211 and 332.  See that they are redundant.  That means that on THOSE days, we chose not to work.  This gives us an intuition.   If we end up storing the days that we worked or didn't, we'd be able to "recover" the solution from the result.   We can do so by looking for successive values that are not changed, but it is also easy enough to simply store that we did or didn't choose the job for the day in another array.



In [ ]:
def job_selection_w_recover(calendar):
    size = len(calendar)
    j = [0] * size
    s = [False] * size
    j[0] = calendar[0]
    s[0] = True
    j[1] = max(calendar[0], calendar[1])
    s[1] = calendar[1] > calendar[0]

    for i in range(2, size):
        with_last = j[i-2] + calendar[i]
        with_out_last = j[i-1]
        j[i] = max(with_last, with_out_last)
        s[i] = with_last > with_out_last


    return s, j

s, j = job_selection_w_recover(p)

print(s)
print(j)

## Reading recovery Array

We can see a series of True or Falses for each day.  True means we took the job, false means we didn't.

**This is not the list of days we worked to earn maximum money** because remember we can't work consecutive days?

If we start at the end of this list though... we can see that we indeed worked the last day.  That means we didn't work the day before, so we can skip that day (move back 2 elements) and see that we didn't accept that day...  so we only need to step back one day.  

In short, iterate backward from the end, skipping Two days for True and One day for false.  If the day you read is True, add that index to the days worked, otherwise, skip.

In [ ]:
def job_selection_recover(job_solution):
    a = []
    i = len(job_solution) - 1

    while i >= 0:
        if job_solution[i]:
            a.append(i)
            i -= 2
        else:
            i -= 1
    return a

job_selection_recover(s)

## Results of Recovery
We see this list that we worked the 13th day, 10th day, 8th day... 0th day.

We then combine this function with the original function so that can make one call as a user to get both the Dollars and the days worked:


In [ ]:
def job_selection_w_recover(calendar):
    size = len(calendar)
    j = [0] * size
    s = [False] * size
    j[0] = calendar[0]
    s[0] = True
    j[1] = max(calendar[0], calendar[1])
    s[1] = calendar[1] > calendar[0]

    for i in range(2, size):
        with_last = j[i-2] + calendar[i]
        with_out_last = j[i-1]
        j[i] = max(with_last, with_out_last)
        s[i] = with_last > with_out_last

    days_worked = job_selection_recover(s)
    return days_worked, j[-1]

days, dollars = job_selection_w_recover(p)

print(f"By Working days: {days} you can earn {dollars} dollars.")


## Alternatives to using Recover?
We show this recover because it is clever and effective, and gives the student some practice at solving problems that aren't directly intuitive.  There are however alternatives to having to use recover.

Think back to the solution of the change making algorithm, where we stored the values of the days selected in each step...  we can do something similar with this result.

At each step rather than storing True/False if we picked that day or not, we either store the days worked from the i-2 day and append the current day to the solution... or store the solution from the previous day.

In [ ]:
def job_selection_no_recover(calendar):
    size = len(calendar)
    j = [0] * size
    days_worked = [None] * size  # CAN ALSO MAINTAIN THE DAYS WORKED TO NOT REQUIRE THE RECOVERY (LIKE CHANGE MAKING)
    j[0] = calendar[0]
    days_worked[0] = [0]
    j[1] = max(calendar[0], calendar[1])
    days_worked[1] = [1 if calendar[1] > calendar[0] else 0]

    for i in range(2, size):
        with_last = j[i-2] + calendar[i]
        with_out_last = j[i-1]
        j[i] = max(with_last, with_out_last)
        if with_last > with_out_last:
            days_worked[i] = days_worked[i-2].copy()
            days_worked[i].append(i)
        else:
            days_worked[i] = days_worked[i-1].copy()
    for l in days_worked:
        print(l)
    return days_worked[-1], j[-1]

days_worked, dollars = job_selection_no_recover(p)
print(f"By Working days: {days_worked} you can earn {dollars}")



# 0-1 Knapsack

This is a famous computer science problem used to demonstrate and introduce students to dynamic programming.  The concept is that you are given a knapsack with finite capacity, and a set of items with different weights and values.  The goal is to fill the knapsack with items such that you maximize the total value of items in the knapsack but remain at or under capacity.

We solve this using dynamic programming by varying the size of both the capacity of the knapack AND the items available to place into it.

Let's make this simple to help step into it.  We will have 3 variables, $w$ (the list of weights), $v$ (the list of values and $c$ the capacity of the knapsack.

In [ ]:
w = [10,4,2]
v = [4,1,5]
c = 15


## Initialization

Let's start with the problem with only one item.  (the one with weight of 10 and value of 4).

When we solve for the different capacities of the knapsack we will insert the first value only when the capacity is large enough to fit in.

In [ ]:
solution = [0] * (c + 1)

for j in range(c+1):
    if w[0] <= j:
        solution[j] = v[0] 
    else:
        solution[j] = 0
        
print(solution)

## Explaning the array we see.

We see 16 values.  Capacity 0 is only used later.
We see that the 11th value (capacity 10) is when 4 is allowed to be placed in the solution.

Now, we want to see what it will look like with a second item.  in order to do this, we need a second row in the array... so we're going to do this manually just for demonstration purposes, and then make it generic.

In [ ]:
solution2 = []
solution2.append(solution)
solution2.append([0] * (c+1))

for s in solution2:
    print(s)

We are going to apply a second loop, but instead of only looking at the size (that is part of it) we have to see if the previous solution for that capacity was any better.

Note we are now looking at item 0 with a weight of $w(1)=4$ and value of $v(1) = 1$

In [ ]:
for j in range(c+1):
    if w[1] <= j:   # it will fit, so look to see if it's any better
        with_item = solution2[0][j - w[1]] + v[1]
    else:
        with_item = -1
    
    without_item = solution2[0][j]
    solution2[1][j] = max(with_item, without_item)
    
for s in solution2:
    print(s)

## Explaining what we see...

We find that at capacity 4, we add the item with value 1.
But once we get to capacity 10 we choose to keep the more valuable item in.

It isn't until we get to the final capacity 15 do we see that the value of capacity of 15-4 (or capacity 9) (which is 4) plus the value of the current item (so 4+1 = 5) is greater than simply looking at the previous row's value (the best value without this item).

## Making it generic

To make it generic, we need to initalize a 2d array.  We still do the first row as a unique run (because there is no previous row to compare it with, (we're dealing with a trival problem of only a single item).

And then for subsequent rows, we loop through each row, looking at the previous row for the best values either WITH or WITHOUT the item represented by the current row.



In [ ]:
import numpy as np
def knapsack(weights, values, capacity):
    
    # THE SOLUTION IS NOW A 2D ARRAY OF SIZE N (NUMBER OF ITEMS) AND C+1 (CAPACITY +1)
    solution = np.zeros((len(weights), capacity+1))

    # INITIALIZE THE FIRST ITEM JUST AS WE DID BEFORE
    for j in range(capacity+1):
        solution[0, j] = values[0] if j >= weights[0] else 0

    # NOW WE LOOP THROUGH EACH ROW, ASSESSING EACH NEW ITEM I
    for i in range(1, len(weights)):
        
        # FOR EACH ITEM I, VARY THE CAPACITY AND COMPARE IT TO THE PREVIOUS ROWS
        for j in range(capacity+1):
            if weights[i] <= j:  # IF THIS ITEM CAN FIT
                with_item = solution[i-1, j-weights[i]] + values[i]
            else:
                with_item = -1

            without_item = solution[i-1,j]
            solution[i, j] = max(without_item, with_item)

    return solution

solution = knapsack(w,v,c)

print(solution)


## Assessing the results, how to recover????

We see here that the solution[-1,-1] will give us 9.   That is the "max value" the knapsack can have...

With our human intuition with this small of a dataset is the 10lb item valued at 4 and the 2lb item valued at 5.   But how can we tell that from this 2D array.

The solution requires a bit of backtracking (as the recovery did for the Job Selection algorithm).

Here is the jist:

- Start at the answer.  If the number directly above it matches, then this item was NOT included.
- Continue Upward until the value above is NOT the same.  If it is the same, it WAS included, add that row's item to the "results"
- Shift columns to the left the weight of the previously added item and repeat.

Alternatively, you can also maintain a second 2D array that stores the selected values (similar to how we managed Change and Job selection).   NOTE because this is a 2D problem... the size cost for storing all of those solutions can be expensive.

# Student Excercise: Implement a Recovery Algorithm


---
<span style="color:green">
This may be tricky, but I'm sure you're up for the challenge.  Implement the solution above using the starter code below.
</span>

---

In [ ]:
def knapsack_recover(solution, weights, values):
    rows, cols = solution.shape

    curr_row = rows - 1
    curr_col = cols - 1

    result = []

    curr_val = solution[curr_row, curr_col]
    while curr_val > 0:
        ### IMPLEMENT THE LOOP, IF YOU FIND A MATCH OF THE PREVIOUS COLUMN, MOVE UP A ROW AND CONTINUE
        ### IF YOU FIND A MISMATCH ADD THE CURRENT ROW TO THE RESULT AND
        ### FROM THE PREVIOUS COLUMN SUBTRACT THE WEIGHT OF THE ITEM FROM THE 
        ### COLUMN AND SUBTRACT THE VALUE FROM CURRENT VALUE

        break  # REMOVE THIS LINE WHEN TRYING OUT YOUR CODE.

    return result

### Testing results

If you implemented the above code correctly, you should get the following results:  [2,0]

In [ ]:
test_result = knapsack_recover(solution, w,v)

print(test_result == [2,0])

## A more elaborate test on a larger knapsack



In [ ]:
w2 = [1,4,2,6,7,4,1,3]
v2 = [0,1,5,6,7,2,4,1]
c2 = 13
solution2 = knapsack(w2,v2,c2)

expected_solution = [6,5,3,2]

test_result2 = knapsack_recover(solution2, w2, v2)
print(test_result2 == [6,5,3,2])